In [23]:
import pandas as pd
import numpy as np

# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
import warnings
warnings.filterwarnings('ignore')


import pandas as pd
import numpy as np
import re

gdp_years = [str(x) for x in range(2006, 2016)]

def clean_names(x):
    nums = re.match('(\D*)\d*', x)
    if nums:
        x = nums.group(1)

    if x == "Republic of Korea": return "South Korea"
    if x == "United States of America": return "United States"
    if x == "United Kingdom of Great Britain and Northern Ireland": return "United Kingdom"
    if x == "China, Hong Kong Special Administrative Region": return "Hong Kong"
    
    pars = re.match('(.*)\(.*\)', x)
    if pars: return pars.group(1).strip()
    
    return x


In [118]:
#GDP = pd.read_csv('assets/gdp.csv')
GDP = pd.read_csv('assets/gdp.csv',  skiprows=4)
#GDP.head()
#GDP.tail(4)
#GDP[30:35]
#GDP.loc[100:105, '2000':'2005']
#GDP[['Country Name', 'Country Code']]
#GDP['Population'] = GDP['2015'].mean()
#GDP
#GDP['1960'] == np.nan
#import math
#math.isnan(np.nan)
#~GDP['1960'].apply(math.isnan)
#GDP[~GDP['1960'].isnull()]

#GDP.T
# Multilevel index, stack, unstack, 

GDP['Country Name'].replace(["Korea, Rep.", "Iran, Islamic Rep.", "Hong Kong SAR, China"], 
                            ["South Korea", "Iran", "Hong Kong"],
                            inplace=True)
GDP.rename({'Country Name': 'Country'}, inplace=True, axis='columns')
GDP = GDP.filter(items=['Country'] + gdp_years)


In [119]:


energy = pd.read_excel('assets/energy_indicators.xls',
                           skiprows=17, 
                           skipfooter=38,
                           na_values=['...']).iloc[:, 2:]
energy.rename({'Unnamed: 2': 'Country',
               'Petajoules': 'Energy Supply',
               'Gigajoules': 'Energy Supply per Capita',
               '%': '% Renewable'}, inplace=True, axis='columns')
energy['Energy Supply'] = energy['Energy Supply'].apply(lambda x: x * 1_000_000)
energy['Country'] = energy['Country'].apply(clean_names)


science = pd.read_excel('assets/sci_journal_rank.xlsx')


In [120]:


def merged():  
    return science[:15].merge(energy, how='inner', on='Country').merge(GDP, how='inner', on='Country').set_index('Country')


merged()

#merged().groupby('Region').first()
#merged().groupby('Region').describe()
merged().groupby('Region').to_frame()


AttributeError: 'DataFrameGroupBy' object has no attribute 'to_frame'

In [116]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    top15 = merged()
    ax = top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#e41a1c','#4daf4a','#4daf4a','#4daf4a','#377eb8',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#4daf4a','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(top15.index):
        ax.annotate(txt, [top15['Rank'][i], top15['% Renewable'][i]], ha='center')

    print("A bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

plot_optional()

KeyError: 'Country'